In [ ]:
#inicializando GEE
try:
    import ee
except ModuleNotFoundError: 
    !pip install earth-engine-api
    import ee

try:
    ee.Initialize()
except ee.EEException:
    !earthengine authenticate

import geemap
import folium
from ipygee import *
import matplotlib.pyplot as plt
from matplotlib import gridspec
import pandas as pd

In [ ]:
#polygons loading hardcode
#id467 = r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_scripts\source\ID_467.shp"
#id467_MODIS = r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_scripts\source\ID_467_MODIS_1.shp"
campo_abr22 = r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02h_dados_scripts\source\20220311_dataset_campo_abr22_f_WGS84_Z.shp"


#create ee.Feature with GEEMAP
pol = geemap.shp_to_ee(campo_abr22)

#Visualization with Folium Map GEE integrated
def add_ee_layer(self, ee_object, vis_params, name):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            fill = False,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    except:
        print("Could not display {}".format(name))

#Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

polygon_params = {
  'palette': ['red']}
location = pol.geometry().centroid(10).coordinates().getInfo()[::-1]
Map = folium.Map(location=location, zoom_start=18)
basemaps['Google Satellite Hybrid'].add_to(Map)
Map.add_ee_layer(pol, polygon_params, 'polygon')
loc = '-'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)  
Map.get_root().html.add_child(folium.Element(title_html))
Map

In [ ]:
#NDVI to Landsat-8
def ndvi_func_LC8(image):
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

#NDVI to Sentinel-2
def ndvi_func_S2(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

#Modis scale factor function
def scale_factor_MODIS(image):
    # scale factor for the MODIS MOD13Q1 product
    return image.multiply(0.0001).copyProperties(image, ['system:time_start'])

##Mascará de nuvens para a banda pixel_qa SR landsat 4,5 e 7
def cloudMaskL457(image):
    qa = image.select('pixel_qa') ##substitiu a band FMASK
    cloud1 = qa.bitwiseAnd(1<<5).eq(0)
    cloud2 = qa.bitwiseAnd(1<<7).eq(0)
    cloud3 = qa.bitwiseAnd(1<<3).eq(0)
    mask2 = image.mask().reduce(ee.Reducer.min());
    return image.updateMask(cloud1).updateMask(cloud2).updateMask(cloud3).updateMask(mask2).divide(10000).copyProperties(image, ["system:time_start"])

# Define a cloud masking function.
def maskL8sr(image):
    cloudShadowBitMask = 1<<3
    cloudBitMask = 1<<5
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudBitMask).eq(0))
    return image.updateMask(mask)

def get_s2_sr_cld_col(polygon, start_date, end_date):
    CLOUD_FILTER = 90
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(polygon)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(polygon)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cld_shdw_mask(img):
    BUFFER = 20
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

def add_cloud_bands(img):
    CLD_PRB_THRESH = 40
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
   
    NIR_DRK_THRESH = 0.15
    CLD_PRJ_DIST = 2
   
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

#plot all timeseries
def TS_NDVI_plot(imagecoll, polygon, filter_param, scale):
    #time series
    polygon_ndvi = chart.Image.series(**{'imageCollection': imagecoll.select('NDVI'),
                                       'region': polygon,
                                       'reducer': ee.Reducer.mean(),
                                       'scale': scale,
                                       'xProperty': 'system:time_start'})
    df = polygon_ndvi.dataframe
    #df = df.rolling(filter_param, min_periods=1).mean()
    return df

def plot_LC8_S2_MODIS(ts1,ts2,ts3, ts1_name, ts2_name, ts3_name, fig_title, text):
    #Plot three TS - LANDSAT8/S2/MODIS
    fig, axs = plt.subplots(3, figsize=(10, 15))
    #fig.subplots_adjust(top=0.8)
    
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

    # place a text box in upper left in axes coords
    axs[0].text(0.05, 0.95, text, transform=axs[0].transAxes, fontsize=10,
        verticalalignment='bottom', bbox=props)
    
    plt.suptitle(fig_title, fontsize=14)
                 #, y=0.91)
    
    #ts1
    axs[0].plot(ts1.index, ts1['NDVI'],
            color='green')
    axs[0].set_title(ts1_name, size = 12)
    axs[0].set_ylim(0.0, 1.0)
    axs[0].spines['right'].set_visible(False)
    axs[0].spines['top'].set_visible(False)

    #ts2
    axs[1].plot(ts2.index, ts2['NDVI'],
            color='green')
    axs[1].set_title(ts2_name, size = 12)
    axs[1].set_ylim(0.0, 1.0)
    axs[1].spines['right'].set_visible(False)
    axs[1].spines['top'].set_visible(False)
   
    #ts3
    axs[2].plot(ts3.index, ts3['NDVI'],
            color='green')
    axs[2].set_title(ts3_name, size = 12)
    axs[2].set_ylim(0.0, 1.0)
    axs[2].spines['right'].set_visible(False)
    axs[2].spines['top'].set_visible(False)
    fig.tight_layout(rect=[0, 0.03, 1, 0.98])
    gs = gridspec.GridSpec(29, 21, wspace=0.1, hspace=0.1)
    gs.tight_layout(fig, pad=0)
    return fig

# #collections 2015 - 2021
# MOD13Q1 = (ee.ImageCollection('MODIS/006/MOD13Q1')
#                 .filterBounds(pol)
#                 .filterDate(ee.DateRange('2000-02-10','2022-12-31'))
#                 .sort('system:time_start'))

# # Image collection build, spatial and temporal filter
# l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
#   .filterBounds(pol) \
#   .filterDate('2015-01-01', '2022-12-31').map(maskL8sr)

# #Sentinel-2 TS filter
# s2_sr_cld_col = get_s2_sr_cld_col(polygon=pol, start_date='2017-01-01', end_date='2022-12-31')
# s2_sr = (s2_sr_cld_col.map(add_cld_shdw_mask)
#                              .map(apply_cld_shdw_mask).map(ndvi_func_S2))
#                              #.median())

# #map indices - apply ndvi_func
# images_ndvi_LC8 = l8sr.map(ndvi_func_LC8)
# #images_ndvi_S2 = s2_sr_median.map(ndvi_func_S2)

# #NDVI selection and scale factor to MODIS
# ndvi_modis = MOD13Q1.select('NDVI')

# # mapping function to multiply by the scale factor
# images_ndvi_modis_scaled = ndvi_modis.map(scale_factor_MODIS)

#TS Constructors
# LC8_NDVI_ts = TS_NDVI_plot(images_ndvi_LC8, pol, filter_param =2, scale=30)
# S2_NDVI_ts = TS_NDVI_plot(s2_sr, pol, filter_param =2, scale=10)
# MOD13Q1_NDVI_ts = TS_NDVI_plot(images_ndvi_modis_scaled, pol, filter_param=1, scale=250)

# Figura_TS = plot_LC8_S2_MODIS(LC8_NDVI_ts, S2_NDVI_ts, MOD13Q1_NDVI_ts, ts1_name = 'LANDSAT-8 OLI NDVI TIME SERIES',
#                   ts2_name = 'SENTINEL-2 MSI NDVI TIME SERIES', ts3_name = 'MODIS MOD13Q1 NDVI TIME SERIES',
#                               fig_title='SAF ID 467 - 2010 abandonado?',tecnica='-')

In [ ]:
import geopandas as gpd
import time
#polygons loading hardcode
#poligonos_restauro = r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_shp\20220307_restauro_sabrina\v1\restauro_udps_joined_v1_WGS.shp"
campo_abr22 = r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02h_dados_scripts\source\20220311_dataset_campo_abr22_f_WGS84_Z.shp"

campo_abr22 = gpd.read_file(campo_abr22) 
campo_abr22.columns


In [ ]:
def polygons_NDVI_exports_restauracao(gdf):
        for counter, feature, id_restaur, id_imovel, tecnica_re in zip(gdf.index, gdf['geometry'], gdf['ID_RESTAUR'], gdf['NM_PRODTOR'],  gdf['TECNICA_RE']):
            geojson_data = feature.__geo_interface__
            geometry = ee.Geometry(geojson_data)
            print(counter+1)
            #print(id_imovel, id_restaur, tecnica_re)
            fig_title = 'FIG TIME SERIES '+str(id_imovel)+' '+str(id_restaur)+' '+str(counter)+' .png'
            tecnica_text = 'Técnica de restauração: '+str(tecnica_re)
            print('fig title:', fig_title)
            #TS Constructors
            #collections 2015 - 2021
            MOD13Q1 = (ee.ImageCollection('MODIS/006/MOD13Q1')
                            .filterBounds(geometry)
                            .filterDate(ee.DateRange('2010-01-01','2022-12-31'))
                            .sort('system:time_start'))

            # Image collection build, spatial and temporal filter
            l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
              .filterBounds(geometry) \
              .filterDate('2015-01-01', '2022-12-31').map(maskL8sr)

            #Sentinel-2 TS filter
            s2_sr_cld_col = get_s2_sr_cld_col(polygon=geometry, start_date='2015-01-01', end_date='2022-12-31')
            s2_sr = (s2_sr_cld_col.map(add_cld_shdw_mask)
                                         .map(apply_cld_shdw_mask).map(ndvi_func_S2))
                                         #.median())

            #map indices - apply ndvi_func
            images_ndvi_LC8 = l8sr.map(ndvi_func_LC8)
            #images_ndvi_S2 = s2_sr_median.map(ndvi_func_S2)

            #NDVI selection and scale factor to MODIS
            ndvi_modis = MOD13Q1.select('NDVI')

            # mapping function to multiply by the scale factor
            images_ndvi_modis_scaled = ndvi_modis.map(scale_factor_MODIS)
            
            LC8_NDVI_ts = TS_NDVI_plot(images_ndvi_LC8, geometry, filter_param =1, scale=30)
            S2_NDVI_ts = TS_NDVI_plot(s2_sr, geometry, filter_param =1, scale=10)
            MOD13Q1_NDVI_ts = TS_NDVI_plot(images_ndvi_modis_scaled, geometry, filter_param=1, scale=250)
            
            Figura_TS = plot_LC8_S2_MODIS(LC8_NDVI_ts, S2_NDVI_ts, MOD13Q1_NDVI_ts, ts1_name = 'LANDSAT-8 OLI NDVI TIME SERIES',
                  ts2_name = 'SENTINEL-2 MSI NDVI TIME SERIES', ts3_name = 'MODIS MOD13Q1 NDVI TIME SERIES', 
                                fig_title=fig_title, text=tecnica_text)
            time.sleep(20)
            Figura_TS.savefig(fname=fig_title)
            del LC8_NDVI_ts, S2_NDVI_ts,MOD13Q1_NDVI_ts, Figura_TS

polygons_NDVI_exports_restauracao(campo_abr22)

In [ ]:
campo_abr22 = r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02h_dados_scripts\source\20220311_dataset_campo_abr22_f_WGS84_Z.shp"
campo_abr22 = gpd.read_file(campo_abr22) 
campo_abr22.columns

In [ ]:
campo_abr22.head()

In [ ]:
def polygons_NDVI_exports_saf(gdf):
        for counter, feature, id_plantio, ano_impl in zip(gdf.index, gdf['geometry'], gdf['ID_PLANTIO'], gdf['ANO_IMPL']):
            geojson_data = feature.__geo_interface__
            geometry = ee.Geometry(geojson_data)
            print(counter+1)
            #print(id_imovel, id_restaur, tecnica_re)
            fig_title = 'FIG TIME SERIES '+str(id_plantio)+' '+str(counter)+' .png'
            ano_impl_text = 'ANOIMPL: '+str(ano_impl)
            print('fig title:', fig_title)
            #TS Constructors
            #collections 2015 - 2021
            MOD13Q1 = (ee.ImageCollection('MODIS/006/MOD13Q1')
                            .filterBounds(geometry)
                            .filterDate(ee.DateRange('2000-01-01','2022-12-31'))
                            .sort('system:time_start'))

            # Image collection build, spatial and temporal filter
            l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
              .filterBounds(geometry) \
              .filterDate('2015-01-01', '2022-12-31').map(maskL8sr)

            #Sentinel-2 TS filter
            s2_sr_cld_col = get_s2_sr_cld_col(polygon=geometry, start_date='2017-01-01', end_date='2022-12-31')
            s2_sr = (s2_sr_cld_col.map(add_cld_shdw_mask)
                                         .map(apply_cld_shdw_mask).map(ndvi_func_S2))
                                         #.median())

            #map indices - apply ndvi_func
            images_ndvi_LC8 = l8sr.map(ndvi_func_LC8)
            #images_ndvi_S2 = s2_sr_median.map(ndvi_func_S2)

            #NDVI selection and scale factor to MODIS
            ndvi_modis = MOD13Q1.select('NDVI')

            # mapping function to multiply by the scale factor
            images_ndvi_modis_scaled = ndvi_modis.map(scale_factor_MODIS)
            
            LC8_NDVI_ts = TS_NDVI_plot(images_ndvi_LC8, geometry, filter_param =1, scale=30)
            S2_NDVI_ts = TS_NDVI_plot(s2_sr, geometry, filter_param =1, scale=10)
            MOD13Q1_NDVI_ts = TS_NDVI_plot(images_ndvi_modis_scaled, geometry, filter_param=1, scale=250)
            
            Figura_TS = plot_LC8_S2_MODIS(LC8_NDVI_ts, S2_NDVI_ts, MOD13Q1_NDVI_ts, ts1_name = 'LANDSAT-8 OLI NDVI TIME SERIES',
                  ts2_name = 'SENTINEL-2 MSI NDVI TIME SERIES', ts3_name = 'MODIS MOD13Q1 NDVI TIME SERIES', 
                                fig_title=fig_title, text=ano_impl_text)
            time.sleep(20)
            Figura_TS.savefig(fname=fig_title)
            del LC8_NDVI_ts, S2_NDVI_ts,MOD13Q1_NDVI_ts, Figura_TS

polygons_NDVI_exports_saf(campo_abr22)

In [ ]:
# def plot_getURL_GEE_NDVI(periods_NDVI, collections_NDVI, area):
#         images = []
#         for period, collection in zip(periods_NDVI, collections_NDVI):
#         #MAP NDVI Function
#             #NDVI_collection  = collection.map(NDVI_function)
#             # get the NDVI max()
#             #collection_ndvi_max = ee.Image(NDVI_collection.select('ndvi').max())
#             #reproject image to apply to process in gee (burnout)
#             collection_ndvi_max_reprojected = collection_ndvi_max.reproject(crs=ee.Projection('EPSG:3395'), scale = 10)
#             #geometry to fc to processing
#             fc = ee.FeatureCollection(area)
#             #transform polygon to raster
#             polygon_raster = (ee.Image().paint(**{
#               'featureCollection': fc.geometry(),
#               'color': '0000FF',
#               'width': 5
#             }))
#             # Get final visualization thumbnail URL.
#             vis_img = collection_ndvi_max_reprojected.visualize(**{'min': 0, 'max': 1, 'palette': ['red', 'yellow','green']});
#             vis_img = vis_img.blend(polygon_raster).clipToCollection(fc)
#             #get url
#             url = vis_img.getThumbURL({'dimensions': 1024, 'region': fc.geometry()})
#             # Get image content from the URL.
#             response = requests.get(url)
#             image = Image.open(BytesIO(response.content))
#             images.append(image)
#         return images

# def figure_url_plot(periods, images, name):
#         fig  = plt.figure(figsize=(15,20))
#         for i, image in zip(range(1, 7), range(0, len(images))):
#             plt.subplot(2, 3, i)
#             plt.imshow(images[image])
#             plt.axis('off')
#             plt.title(periods[image], fontsize = 12)
#         fig.suptitle('NDVI - '+ str(name), fontsize=20)
#         norm = mpl.colors.Normalize(vmin=0, vmax=1)
#         plt.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap="RdYlGn"), fraction=0.046, pad=0.04, orientation='vertical', label='NDVI')
#         fig.tight_layout()
#         return fig

# def get_periods_and_collection_items_S2(period_NDVI_dict, area):
#         #loop through dates in dict
#         periods = list()
#         collections = list()
#         for period in period_NDVI_dict:
#             start_date = period_NDVI_dict[period]['start_date']
#             end_date = period_NDVI_dict[period]['end_date']
#             #print(period, start_date, end_date)
#             #Sentinel-2 TS filter
#             s2_sr_cld_col = get_s2_sr_cld_col(polygon=area, start_date='2015-01-01', end_date='2022-12-31')
#             s2_sr = (s2_sr_cld_col.map(add_cld_shdw_mask)
#                                          .map(apply_cld_shdw_mask).map(ndvi_func_S2)
#                                          .median())
#             #MAP NDVI
#             #collection_NDVI_LC8  = collection.map(NDVI_function_LC8)
#             #NDVI_img_max_clipped = ee.Image(collection_NDVI_LC8.max().clip(area))
            
#             periods.append(period)
#             collections.append(s2_sr)

#         return periods, collections    
# crop_interest_period_NDVI_2017_2019 = {
#                                     '2017': {'start_date':"2017-01-01",'end_date':"2017-12-31"},
#                                     '2018': {'start_date':"2018-01-01",'end_date':"2018-12-31"},
#                                     '2019': {'start_date':"2019-01-01",'end_date':"2019-12-31"},
#                                     '2020': {'start_date':"2020-01-01",'end_date':"2020-12-31"},
#                                     '2021': {'start_date':"2021-01-01",'end_date':"2021-12-31"},
#                                     '2022': {'start_date':"2022-01-01",'end_date':"2022-12-31"}                                  
#                                     }
# periods, collections = get_periods_and_collection_items_S2(period_NDVI_dict=crop_interest_period_NDVI_2017_2019, area=pol)
# #list_crops_images_NDVI = get_NDVI_imagesURL(crop_interest_period_NDVI_2013_2021_S2, df=crops_df)


 